In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
dataset=pd.read_csv('Social_Network_Ads.csv')

In [4]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset=dataset.drop("User ID",axis=1)

In [8]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [10]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [11]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [12]:
indep=dataset[['Age', 'EstimatedSalary',  'Gender_Male']]

In [13]:
dep=dataset[['Purchased']]

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [18]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [19]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [20]:
param_grid={'criterion':['gini', 'entropy', 'log_loss'],'splitter':['best', 'random'],'max_features':['sqrt', 'log2']}

In [24]:
grid=GridSearchCV(DecisionTreeClassifier(),param_grid,refit=True,verbose=3,n_jobs=1,scoring='f1_weighted')

In [25]:
grid=grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.805 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.844 total time=   0.0s
[CV 3/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.796 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.852 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.925 total time=   0.0s
[CV 1/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.771 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.774 total time=   0.0s
[CV 3/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.794 total time=   0.0s
[CV 4/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.888 total time=   0.0s
[CV 5/5] END criterion=gini, max_features=sqrt, splitter=random;, score=0.79

In [26]:
re=grid.cv_results_

In [27]:
re
print("The R_score value for best parameter {}:".format(grid.best_params_))

The R_score value for best parameter {'criterion': 'log_loss', 'max_features': 'log2', 'splitter': 'best'}:


In [28]:
grid_predictions=grid.predict(x_test)

In [29]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [30]:
cm

array([[76,  9],
       [10, 39]], dtype=int64)

In [31]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [32]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.88      0.89      0.89        85
           1       0.81      0.80      0.80        49

    accuracy                           0.86       134
   macro avg       0.85      0.85      0.85       134
weighted avg       0.86      0.86      0.86       134



In [33]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.8450180072028812

In [34]:
table=pd.DataFrame.from_dict(re)

In [35]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005326,0.004591,0.018873,0.015222,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.804764,0.843702,0.796284,0.851527,0.925272,0.844310,0.045777,5
1,0.000000,0.000000,0.006371,0.007803,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.770712,0.773585,0.793565,0.887907,0.795256,0.804205,0.043032,12
2,0.000000,0.000000,0.003193,0.006386,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.864871,0.808392,0.773585,0.831098,0.883278,0.832245,0.039192,7
3,0.001201,0.002402,0.010370,0.009374,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.782557,0.870898,0.778179,0.811321,0.886792,0.825949,0.044947,9
4,0.001596,0.001356,0.003577,0.003688,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.808392,0.758030,0.870362,0.865054,0.833863,0.044350,6
5,0.006317,0.007737,0.000000,0.000000,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.826263,0.808392,0.796687,0.791152,0.901744,0.824848,0.040285,10
6,0.000401,0.000802,0.009075,0.006795,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.829615,0.906252,0.832483,0.851527,0.847020,0.853379,0.027716,2
7,0.000000,0.000000,0.012746,0.006804,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.864871,0.831253,0.796687,0.850543,0.770530,0.822777,0.034713,11
8,0.002349,0.001192,0.005975,0.002170,log_loss,sqrt,best,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.804764,0.831253,0.815056,0.870362,0.905069,0.845301,0.037298,4
9,0.000274,0.000547,0.000000,0.000000,log_loss,sqrt,random,"{'criterion': 'log_loss', 'max_features': 'sqr...",0.887346,0.864635,0.735849,0.851527,0.887907,0.845453,0.056525,3


In [36]:
grid.predict([[29,29000,1]])

array([1], dtype=int64)